### "Streaming" : 長い応答のストリーミング受信

In [1]:
#Install dependencies
# %uv pip install anthropic python-dotenv

In [2]:
# Load env variables
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# Create an API client
from anthropic import Anthropic

client = Anthropic()
model = "claude-sonnet-4-0"

### 既存の会話ライブラリ

In [4]:
def add_user_message(message, text):
    user_message = {
        "role": "user",
        "content": text
    }
    message.append(user_message)

def add_assistant_message(message, text):
    assistant_message = {
        "role": "assistant",
        "content": text
    }
    message.append(assistant_message)

# Make a request
def chat(messages, system=None,temperature=1.0):
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature
    }
    if system:
        params["system"] = system

    message = client.messages.create(**params)
    return message.content[0].text


### 長い会話の例：架空のデータベースの生成

In [5]:
# Make an initial list of messages
messages = []

add_user_message(
    messages,
    "Write a 1 sentence desctiption of a fake database. Answer with Japanese"
)
print(messages[-1]['content'])

stream = client.messages.create(
    model=model,
    max_tokens=1000,
    messages=messages,
    stream=True
)
for event in stream:
    print(event)


Write a 1 sentence desctiption of a fake database. Answer with Japanese
RawMessageStartEvent(message=Message(id='msg_0123xvGyfQvtPf2eG34gERQ6', content=[], model='claude-sonnet-4-20250514', role='assistant', stop_reason=None, stop_sequence=None, type='message', usage=Usage(cache_creation=CacheCreation(ephemeral_1h_input_tokens=0, ephemeral_5m_input_tokens=0), cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=24, output_tokens=1, server_tool_use=None, service_tier='standard')), type='message_start')
RawContentBlockStartEvent(content_block=TextBlock(citations=None, text='', type='text'), index=0, type='content_block_start')
RawContentBlockDeltaEvent(delta=TextDelta(text='偽のデータベースは', type='text_delta'), index=0, type='content_block_delta')
RawContentBlockDeltaEvent(delta=TextDelta(text='、実', type='text_delta'), index=0, type='content_block_delta')
RawContentBlockDeltaEvent(delta=TextDelta(text='在', type='text_delta'), index=0, type='content_block_delta')
RawContentBlo

### with ... stream ... as を使ったストリーム受信

In [6]:
# Make an initial list of messages
messages = []

add_user_message(
    messages,
    "Write a 1 sentence desctiption of a fake database. Answer with Japanese"
)
print(messages[-1]['content'])

with client.messages.stream(
    model=model,
    max_tokens=1000,
    messages=messages
) as stream:
    for text in stream.text_stream:
        print(text, end="", flush=True)


Write a 1 sentence desctiption of a fake database. Answer with Japanese
偽のデータベースは、架空の顧客情報や製品データなどの実在しない情報を含む、テスト目的や開発環境で使用されるシミュレーション用のデータ集合体です。

In [7]:
# Make an initial list of messages
messages = []

add_user_message(
    messages,
    "Write a 1 sentence desctiption of a fake database. Answer with Japanese"
)
print(messages[-1]['content'])

with client.messages.stream(
    model=model,
    max_tokens=1000,
    messages=messages
) as stream:
    for text in stream.text_stream:
        # print(text, end="", flush=True)
        pass

stream.get_final_message()

Write a 1 sentence desctiption of a fake database. Answer with Japanese


ParsedMessage(id='msg_01HUcWVBdNbTXi6kUo8KoBzb', content=[ParsedTextBlock(citations=None, text='偽の顧客データベースには、架空の名前、住所、電話番号、購入履歴などの虚偽情報が含まれている。', type='text', parsed_output=None)], model='claude-sonnet-4-20250514', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(cache_creation=CacheCreation(ephemeral_1h_input_tokens=0, ephemeral_5m_input_tokens=0), cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=24, output_tokens=48, server_tool_use=None, service_tier='standard'))